In [ ]:
!pip install cartopy

In [ ]:
!pip install geopandas
!pip install --no-binary shapely shapely --force

In [ ]:
import os
import subprocess
import re

In [ ]:
!wget https://gist.githubusercontent.com/wxguy/945d18af773c2208a33a5e0429a4b813/raw/448446a0561f81ac9e2c65fb3d1214d647ed17ad/wgrib2_auto_compile_script.sh

In [ ]:
os.system('sh wgrib2_auto_compile_script.sh')

In [ ]:
!wget https://www.ncei.noaa.gov/pub/has/model/HAS012390798/ncep_global_ssi.2022120100.tar

In [ ]:
!wget https://www.ncei.noaa.gov/pub/has/model/HAS012390798/ncep_global_ssi.2022120106.tar

In [ ]:
!wget https://www.ncei.noaa.gov/pub/has/model/HAS012390798/ncep_global_ssi.2022120112.tar

In [ ]:
!wget https://www.ncei.noaa.gov/pub/has/model/HAS012390798/ncep_global_ssi.2022120118.tar

In [ ]:
!tar xvf ncep_global_ssi.2022120100.tar

In [ ]:
!find . ! -name '*pgrb2.0p25.f*' -type f -delete
!find .  -name '*pgrb2.0p25.f00[6-9]' -type f -delete


In [ ]:
dir_path = "./"
file_pattern = "gdas.t00z.pgrb2.0p25.f0"

for filename in os.listdir(dir_path):
    if filename.startswith(file_pattern):
        comando = f"wgrib2 {os.path.join(dir_path, filename)} -d 1 -v"
        saida = subprocess.check_output(comando, shell=True).decode("utf-8").strip()
        print(saida)
        padrao = r"d=(\d{10})"


        resultado = re.search(padrao, saida)
        parts = saida.split(':')
        # Mapear as previsões aos valores correspondentes
        print(parts[-2])
        forecast_mapping = {
          "anl": "00",
          "1 hour fcst": "01",
          "2 hour fcst": "02",
          "3 hour fcst": "03",
          "4 hour fcst": "04",
          "5 hour fcst": "05",
           }
           # Verificar se a última parte corresponde a uma previsão conhecida
        if parts[-2] in forecast_mapping:

              forecast_value = forecast_mapping[parts[-2]]
              print(f"A previsão é: {forecast_value}")
        else:
             print("Previsão desconhecida.")


        if resultado:
            data = resultado.group(1)
            data = data[:-2] + forecast_value
            filename_out="gdas."+data+".grib2"
            print('agora',data,filename_out)
            comando = f"wgrib2 {os.path.join(dir_path, filename)} -new_grid_winds grid -new_grid latlon 280:180:.25 -40:120:.25 "+filename_out
            print(comando)
            saida = subprocess.check_output(comando, shell=True).decode("utf-8").strip()
        else:
            print("Nenhuma data encontrada na expressão.")



In [ ]:

!ls -l

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Definir a área de interesse
lat_start, lat_end, lat_step = -40, -10, 0.25
lon_start, lon_end, lon_step = 285, 325, 0.25

# Criar a projeção e o objeto de figura
proj = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': proj})

# Definir os limites da área de interesse
ax.set_extent([lon_start, lon_end, lat_start, lat_end], crs=proj)

# Adicionar os contornos dos continentes e países
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=0.5)

# Adicionar um quadrado para destacar a área
ax.plot([lon_start, lon_end, lon_end, lon_start, lon_start],
        [lat_start, lat_start, lat_end, lat_end, lat_start],
        color='red', linewidth=2, transform=proj)

# Exibir o mapa
plt.show()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp *.grib2 /content/drive/MyDrive/gis4wrf/dataset/.

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

In [ ]:
!rm gdas*